<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# NLP Using the Twitter API: Guided Lab



---


<img src="https://snag.gy/RNAEgP.jpg" width="600">

### Can we correctly identify which of these two old men tweeted what?

> *Note: this lab is intended to be a guided lab until the independent practice questions.*


## Goals
---

We are going to attempt to classify whether a tweet comes from Trump or Sanders.  This lab involves multiple steps:
- Create a developer account on Twitter
- Create a method to pull a list of tweets from the Twitter API
- Perform proper preprocessing on our text
- Engineer sentiment feature in our dataset using TextBlob
- Explore supervised classification techniques


## Twitter API Developer Registration
---

If you haven't registered a Twitter account yet, this is a requirement in order to have a "developer" account.

[Twitter Rest API](https://dev.twitter.com/rest/public)



## Create an "App"

---

![](https://snag.gy/HPBQbJ.jpg)

Go to Twitter and register an "app" [apps.twitter.com](https://apps.twitter.com/).

> **Note**: For the required website field you can put a placeholder.

After you set up our app, you will only need to reference the cooresponding keys Twitter generates for our app.  These are the keys that we will use with our application to communicate with the Twitter API.

## Install Python Twitter API library

---

Someone was nice enough to build a Python libary for us. It makes pulling tweets simple: we only need to plug in our keys and start collecting data. The library we will be using is provided by [Python Twitter Tools](http://mike.verdone.ca/twitter/).

To install it, uncomment and run the next frame (there is no conda package).

In [1]:
pip install twitter python-twitter

SyntaxError: invalid syntax (<ipython-input-1-3975cdb5d74d>, line 1)

In [2]:
!ls

case-study-twitter-api-nlp.ipynb


## Some Boring Twitter Rules
---

**Twitter notifies you they will rate limit your requests:**

>When using application-only authentication, rate limits are determined globally for the entire application. If a method allows for 15 requests per rate limit window, then it allows you to make 15 requests per window — on behalf of your application. This limit is considered completely separately from per-user limits. https://dev.twitter.com/rest/public/rate-limiting

Here's a quick overview of what Twitter says are "the rules":

![](https://snag.gy/yJ6vIH.jpg)


## About those Keys: OAuth Review
---

![](https://g.twimg.com/dev/documentation/image/appauth_0.png)

## What's going on here?  Take a minute..

## Our Application Keys
---

Take note of your application keys you will use to connect to Twitter and mine tweets from the official Bernie Sanders and Donald Trump twitter accounts:

![](https://snag.gy/H1djQK.jpg)

## `TweetMiner` class structure

---

The following code will get you up and running, providing connectivity to twitter. The class has the ability to make requests and can eventually transform the JSON responses into DataFrames.

This is a great example of using object-oriented Python to organize our code!

> **Note:** "request_limit" is used in this class to limit the number of tweets that are pulled per instance request.  Setting it to something lower until you've worked the bugs out of your request, and captured the data you want, is essential to avoiding the rate limit blocks.

### Twitter API key setup

Fill the information below in with the keys for your account.

- **consumer_key** - Find this in your app page under the "Keys and Access Tokens"
- **consumer_secret** - Right under **consumer_key** in the "Keys and Access Tokens" tab
- **access_token_key** - You will need to click the button to generate tokens to get this
- **access_token_secret** - Also available after you generate tokens


In [4]:
import twitter, re, datetime, pandas as pd
from local_settings import twitter_keys

# your keys go here:
""" twitter_keys = {
    'consumer_key':        'lA6UQplM5sxIAFr83ueUl9sgE',
    'consumer_secret':     'f94t4BD6Vj7aCVkX1qAIVwsP4x69J2vXvm61FTIuwb9GfmdsuP',
    'access_token_key':    '1203210464-FYE1FvoUx1GjVcoyok3U1brWDpELBJEcSNGC1OC',
    'access_token_secret': 'kuVli1j010RPdYTbozFiIyp8QxX6PqRApdi49baLqBgzo'
}"""

api = twitter.Api(
    consumer_key         =   twitter_keys['consumer_key'],
    consumer_secret      =   twitter_keys['consumer_secret'],
    access_token_key     =   twitter_keys['access_token_key'],
    access_token_secret  =   twitter_keys['access_token_secret']
)


In [5]:
class TweetMiner(object):

    result_limit    =   20    
    api             =   False
    data            =   []
    
    def __init__(self, keys_dict, api, result_limit = 20):
        
        self.api = api
        self.twitter_keys = keys_dict
        
        self.result_limit = result_limit
        

    def mine_user_tweets(self, user="dyerrington", mine_rewteets=False, max_pages=5):

        data           =  []
        last_tweet_id  =  False
        page           =  1
        
        while page <= max_pages:
            
            if last_tweet_id:
                statuses   =   self.api.GetUserTimeline(screen_name=user, count=self.result_limit, max_id=last_tweet_id - 1)        
            else:
                statuses   =   self.api.GetUserTimeline(screen_name=user, count=self.result_limit)
                
            for item in statuses:

                mined = {
                    'tweet_id':        item.id,
                    'handle':          item.user.name,
                    'retweet_count':   item.retweet_count,
                    'text':            item.text,
                    'mined_at':        datetime.datetime.now(),
                    'created_at':      item.created_at,
                }
                
                last_tweet_id = item.id
                data.append(mined)
                
            page += 1
            
        return data

## Instantiate the class
---

Make sure you pass the keys dictionary and the api as arguments.

**Check:** call the object's `mine_user_tweets()` method, providing a user to pull the tweets of.

In [6]:
miner = TweetMiner(twitter_keys, api, result_limit=2)

In [7]:
sanders = miner.mine_user_tweets(user='berniesanders', max_pages=5)
donald = miner.mine_user_tweets(user='realDonaldTrump', max_pages=5)

In [8]:
print sanders[0]

{'handle': u'Bernie Sanders', 'mined_at': datetime.datetime(2017, 4, 28, 15, 17, 48, 818830), 'created_at': u'Fri Apr 28 13:23:36 +0000 2017', 'tweet_id': 857948412540596225, 'text': u'RT @CBSNews: "What concerns me right now is these incredible tax breaks that Pres. Trump and Republican leadership are proposing," @BernieS\u2026', 'retweet_count': 314}


### Convert the tweet ouputs to a pandas DataFrame

> *Hint: this is as easy as passing it to the DataFrame constructor!*

In [13]:
s_df = pd.DataFrame(sanders)
s_df.head()

,created_at,handle,mined_at,retweet_count,text,tweet_id
0,Fri Apr 28 13:23:36 +0000 2017,Bernie Sanders,2017-04-28 15:17:48.818830,314,"RT @CBSNews: ""What concerns me right now is th...",857948412540596225
1,Fri Apr 28 12:06:56 +0000 2017,Bernie Sanders,2017-04-28 15:17:48.818845,3047,"No, @realDonaldTrump. Giving huge tax breaks t...",857929121028075521
2,Thu Apr 27 19:50:56 +0000 2017,Bernie Sanders,2017-04-28 15:17:49.095163,192,RT @CBSThisMorning: TOMORROW ON @CBSThisMornin...,857683503479234561
3,Thu Apr 27 15:33:06 +0000 2017,Bernie Sanders,2017-04-28 15:17:49.095179,320,RT @billmckibben: Watching @SenJeffMerkley @Be...,857618617684426753
4,Wed Apr 26 21:08:43 +0000 2017,Bernie Sanders,2017-04-28 15:17:49.318354,2308,RT @keithellison: On the same day Democrats we...,857340689629237248


##  Create the training data

---

Let's get our "mined" data from the Twitter API.  

1. Mine Trump tweets
- Create a tweet DataFrame
- Mine Sanders tweets
- Append the results to our DataFrame

In [10]:
miner = TweetMiner(twitter_keys, api, result_limit=400)

In [12]:
trump_tweets = miner.mine_user_tweets('realDonaldTrump')

In [14]:
bernie_tweets = miner.mine_user_tweets('berniesanders')

In [15]:
trump_df = pd.DataFrame(trump_tweets)
bernie_df = pd.DataFrame(bernie_tweets)

In [16]:
tweets = pd.concat([trump_df, bernie_df], axis=0)
tweets.shape

(1999, 6)

## Any interesting ngrams going on with Trump?
---

Set up a vectorizer from sklearn and fit the text of Trump's tweets with an ngram range from 2 to 4. Figure out what the most common ngrams are.

> **Note:** It's up to you whether you want to remove stopwords or not. How does keeping or removing stopwords affect the results?

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter

# We can use the TfidfVectorizer to find ngrams for us
vect = TfidfVectorizer(ngram_range=(2,4), stop_words="english")

# Pulls all of trumps tweet text's into one giant string
summaries = "".join(trump_df['text'])
ngrams_summaries = vect.build_analyzer()(summaries)

Counter(ngrams_summaries).most_common(20)

[(u'fake news', 39),
 (u'make america', 33),
 (u'america great', 29),
 (u'watch https', 28),
 (u'make america great', 27),
 (u'united states', 23),
 (u'failing nytimes', 18),
 (u'white house', 17),
 (u'tickets https', 15),
 (u'prime minister', 14),
 (u'maga https', 14),
 (u'news media', 12),
 (u'fake news media', 11),
 (u'president obama', 10),
 (u'looking forward', 10),
 (u'north carolina', 10),
 (u'icymi watch', 10),
 (u'donald trump', 10),
 (u'president elect', 10),
 (u'hillary clinton', 10)]

### Look at the ngrams for Bernie Sanders

In [18]:
# We can use the TfidfVectorizer to find ngrams for us
vect = TfidfVectorizer(ngram_range=(2,4), stop_words="english")

# Pulls all of trumps tweet text's into one giant string
summaries = "".join(bernie_df['text'])
ngrams_summaries = vect.build_analyzer()(summaries)

Counter(ngrams_summaries).most_common(20)

[(u'health care', 76),
 (u'donald trump', 64),
 (u'bernie sanders', 47),
 (u'climate change', 28),
 (u'wall street', 25),
 (u'working families', 25),
 (u'democratic party', 23),
 (u'tax breaks', 21),
 (u'american people', 21),
 (u'working people', 18),
 (u'https rt', 16),
 (u'hillary clinton', 14),
 (u'mr trump', 14),
 (u'young people', 13),
 (u'millions people', 13),
 (u'minimum wage', 13),
 (u'health insurance', 12),
 (u'drug companies', 11),
 (u'middle class', 11),
 (u'fair share', 11)]

## Processing the tweets and building a model

---

To do classfication we will need to convert the tweets into a set of features.

**You will need to:**
- Vectorize input text data.
- Intialize a model (try Logistic regression).
- Train / Predict / cross-validate.
- Evaluate the performance of the model.

> **Bonus:** you may have noticed that there are website links in the tweets. What additional preprocessing steps can you do before building the model?


In [20]:
from textacy.preprocess import preprocess_text

tweet_text = tweets['text'].values
clean_text = [preprocess_text(x, fix_unicode=True, lowercase=True, transliterate=False,
                              no_urls=True, no_emails=True, no_phone_numbers=True, no_currency_symbols=True,
                              no_punct=True, no_accents=True)
              for x in tweet_text]

In [23]:
print tweet_text[0:3]

[ u'We are making tremendous progress with the V. A. There has never been so much done so quickly, and we have just started. We love our VETS!'
 u'RT @foxandfriends: Former President Obama\'s $400K Wall Street speech stuns liberal base; Sen. Warren saying she "was troubled by that" http\u2026'
 u'Today, I signed an Executive Order on Improving Accountability and Whistleblower Protection at the @DeptVetAffairs:\u2026 https://t.co/UQ6VNCyhlj']


In [22]:
print clean_text[0:3]

[u'we are making tremendous progress with the v a there has never been so much done so quickly and we have just started we love our vets', u'rt foxandfriends former president obamas usd400k wall street speech stuns liberal base sen warren saying she was troubled by that http', u'today i signed an executive order on improving accountability and whistleblower protection at the deptvetaffairs url']


In [24]:
import numpy as np
y = tweets['handle'].map(lambda x: 1 if x =='Donald J. Trump' else 0).values
print np.mean(y)

0.499749874937


In [25]:
from sklearn.linear_model import LogisticRegression

tfv = TfidfVectorizer(ngram_range=(1, 4), max_features=2000)
X = tfv.fit_transform(clean_text).todense()
print X.shape

(1999, 2000)


In [26]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(LogisticRegression(), X, y, cv=10)

print accuracies
print np.mean(accuracies)

estimator = LogisticRegression()
estimator.fit(X, y)

[ 0.865       0.925       0.935       0.935       0.905       0.935       0.855
  0.89        0.84        0.84422111]
0.892922110553


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

## Check the predicted probability for a random Sanders and Trump tweet
---

Below are provided a couple of tweets from both Sanders and Trump. I'm sure you can figure out on your own which one is which.

Estimate the predicted probability of being trump for the two tweets.

In [27]:
# Prep our source as TfIdf vectors
source_test = [
    "Demanding that the wealthy and the powerful start paying their fair share of taxes that's exactly what the American people want.",
    "Crooked Hillary is spending tremendous amounts of Wall Street money on false ads against me. She is a very dishonest person!"
]

############
# NOTE:  Do not re-initialize the tfidf vectorizor or the feature space willbe overwritten and
# hence your transform will not match the number of features you trained your model on.
#
# This is why you only need to "transform" since you already "fit" previously
#
####
Xtest = tfv.transform(source_test)

estimator.predict_proba(Xtest)

array([[ 0.82093524,  0.17906476],
       [ 0.25150094,  0.74849906]])

## Independent practice questions

---

### 1. Pull tweets for some new users.

Experiment with using more data.  The API will not like it if you blow through their limits - be careful.  Try to grab only what you need one time, then work on the copy of the objects that are returned.  

> Read the documentation about rate limits and see if you can get enough without hitting the rate limit.  Are there any options available in the API to avoid such a problem?

**Pull tweets for more than two different users of your choice.**

In [11]:
# A:

### 2. Build a multi-class classification model to distinguish between the users.

Try a new type of model than we used before.

In [12]:
# A:

### 3. Make a confusion matrix and classification report.

In [13]:
# A:

### 4. What is the most and least "distinctive" tweets for each user?

To find this, identify the tweet that has the highest (correct) predicted probability of being that user's tweet for each user.

In [14]:
# A: